In [1]:
import numpy as np
import pandas as pd
# Paquetes

# Gurobi Python Interface
from gurobipy import*

/Users/gintriag/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# Data
scenarios = [1,2,3,4]
sec_stage = [1,3]


In [3]:
# Create Model
model = Model('example 2.8')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-19


In [4]:
# Variables
PA = model.addVar(name = "PA",lb=0,ub=50,vtype=GRB.CONTINUOUS)
PB = model.addVars(sec_stage,name = "PB",lb=0,ub=40,vtype=GRB.CONTINUOUS)
Pt1 = model.addVars(sec_stage,name = "Pt1",lb=0,vtype=GRB.CONTINUOUS)
Pt2 = model.addVars(scenarios,name = "Pt2",lb=0,vtype=GRB.CONTINUOUS)

# Model update
model.update()

In [5]:
# Objective
obj = 25*144*PA + 0.4*72*(26*PB[1] + 27*Pt1[1] + 0.4*28*Pt2[1] + 0.6*26*Pt2[2]) + 0.6*72*(26*PB[3] + 23*Pt1[3] + 0.4*24*Pt2[3] + 0.6*22*Pt2[4])
model.setObjective(obj, GRB.MAXIMIZE) # maximize profit


In [6]:
# Constraints
# Power produced in first period cap consist of contract A and pool and is capped by 120 MW
model.addConstrs(PA + Pt1[index] <= 120 for index in sec_stage)
# Power produced in first period cap consist of contract A, B and pool and is capped by 120 MW
model.addConstrs(PA + PB[1] + Pt2[scenario] <= 120 for scenario in scenarios[:2])
model.addConstrs(PA + PB[3] + Pt2[scenario] <= 120 for scenario in scenarios[2:])

{3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [7]:
# Update model
model.update()

# Proceed to optimize
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 9 columns and 16 nonzeros
Model fingerprint: 0x6541feb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 4e+03]
  Bounds range     [4e+01, 5e+01]
  RHS range        [1e+02, 1e+02]
Presolve removed 6 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3248960e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.324896000e+05


In [9]:
# Visualization
rows = [1]
columns = ["PA","PB1","PB3","Pt1_1","Pt1_3","Pt2_1","Pt2_2","Pt2_3","Pt2_4"]
Pi_table = pd.DataFrame(columns=columns, index=rows, data=0.0)

Pi_table.loc[1,"PA"] = np.round(PA.x, 1)
Pi_table.loc[1,"PB1"] = np.round(PB[1].x, 1)
Pi_table.loc[1,"PB3"] = np.round(PB[3].x, 1)
Pi_table.loc[1,"Pt1_1"] = np.round(Pt1[1].x, 1)
Pi_table.loc[1,"Pt1_3"] = np.round(Pt1[3].x, 1)
Pi_table.loc[1,"Pt2_1"] = np.round(Pt2[1].x, 1)
Pi_table.loc[1,"Pt2_2"] = np.round(Pt2[2].x, 1)
Pi_table.loc[1,"Pt2_3"] = np.round(Pt2[3].x, 1)
Pi_table.loc[1,"Pt2_4"] = np.round(Pt2[4].x, 1)
    
Pi_table

,PA,PB1,PB3,Pt1_1,Pt1_3,Pt2_1,Pt2_2,Pt2_3,Pt2_4
1,50.0,0.0,40.0,70.0,70.0,70.0,70.0,30.0,30.0
